In [324]:
import pandas as pd
import json
import numpy as np

In [325]:
with open("trial_task.json", "r") as f:
    data = f.read()
    json_data = json.loads(data)

In [326]:
orders_data = pd.read_json("trial_task.json") 
orders_data.head()

,order_id,warehouse_name,highway_cost,products
0,11973,Мордор,-70,"[{'product': 'ломтик июльского неба', 'price':..."
1,62239,хутор близ Диканьки,-15,"[{'product': 'билет в Израиль', 'price': 1000,..."
2,85794,отель Лето,-50,"[{'product': 'зеленая пластинка', 'price': 10,..."
3,33684,Мордор,-30,"[{'product': 'билет в Израиль', 'price': 1000,..."
4,25824,отель Лето,-75,"[{'product': 'автограф Стаса Барецкого', 'pric..."


In [327]:
d = {'product': [], 'price': [], 'quantity': [], 'order_id': []}
result = pd.DataFrame(data=d)
for i in json_data:
    example = i["products"]
    r = pd.DataFrame(data=example)
    r['order_id'] = [i["order_id"] for _ in range(np.shape(r)[0])]
    result = pd.concat([result, r], ignore_index=True)
result['order_id'] = result['order_id'].astype(int)
result['quantity'] = result['quantity'].astype(int)
result['price'] = result['price'].astype(int)
result.head()

,product,price,quantity,order_id
0,ломтик июльского неба,450,1,11973
1,билет в Израиль,1000,3,11973
2,статуэтка Ленина,200,3,11973
3,билет в Израиль,1000,1,62239
4,зеленая пластинка,10,2,85794


In [328]:
orders_data = orders_data.set_index('order_id').join(result.set_index('order_id'), on='order_id')\
                            .drop(['products'], axis=1)
orders_data['total_cost_for_each_product'] = orders_data['price'] * orders_data['quantity']
orders_data = orders_data.reset_index(inplace=False)
orders_data.head()

,order_id,warehouse_name,highway_cost,product,price,quantity,total_cost_for_each_product
0,11973,Мордор,-70,ломтик июльского неба,450,1,450
1,11973,Мордор,-70,билет в Израиль,1000,3,3000
2,11973,Мордор,-70,статуэтка Ленина,200,3,600
3,62239,хутор близ Диканьки,-15,билет в Израиль,1000,1,1000
4,85794,отель Лето,-50,зеленая пластинка,10,2,20


### 1. Найти тариф стоимости доставки для каждого склада

In [329]:
warehouses = orders_data['warehouse_name'].unique()
print(warehouses)

['Мордор' 'хутор близ Диканьки' 'отель Лето' 'остров невезения'
 'гиперборея']


In [330]:
d1 = {'warehouse_name' : warehouses, 'shipping_cost' : np.zeros(len(warehouses))}
shipping_cost_rate = pd.DataFrame(data=d1)
shipping_cost_rate

,warehouse_name,shipping_cost
0,Мордор,0.0
1,хутор близ Диканьки,0.0
2,отель Лето,0.0
3,остров невезения,0.0
4,гиперборея,0.0


In [331]:
for i in range(len(warehouses)):
    eee = orders_data[orders_data['warehouse_name'] == warehouses[i]].groupby('order_id', as_index=False)\
                    .aggregate({'highway_cost' : 'mean', 'quantity' : 'sum'})\
                        .sum().drop(['order_id'])
    shipping_cost_rate.at[i, 'shipping_cost'] = eee['highway_cost'] / eee['quantity']

In [332]:
shipping_cost_rate  #тариф стоимости доставки для каждого склада

,warehouse_name,shipping_cost
0,Мордор,-10.0
1,хутор близ Диканьки,-15.0
2,отель Лето,-25.0
3,остров невезения,-5.0
4,гиперборея,-20.0


### 2. Найти суммарное количество , суммарный доход , суммарный расход и суммарную прибыль для каждого товара (представить как таблицу со столбцами 'product', 'quantity', 'income', 'expenses', 'profit')